In [ ]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from openai import OpenAI

# 環境変数の取得
load_dotenv("../.env")

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

In [ ]:
# モデル名
MODEL_NAME = "gpt-4o-mini"

# メッセージの設定
message = "言語モデルを使う上でのポイントは"

# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": message},
    ]
)

In [ ]:
"""
OpenAI ChatCompletion レスポンス可視化（Notebook向け）

概要:
    OpenAI API のレスポンスを Notebook 上で読みやすく表示するユーティリティです。
    response.model_dump() 可能なオブジェクト、または dict を入力として受け取り、
    以下の順で表示します。

入力（Input）:
    response_obj: Any | dict
        - OpenAI のレスポンスオブジェクト（model_dump() を持つもの）または dict。
        - 例:
            response_obj = response
            response_obj = response.model_dump()

出力（Output）:
    戻り値: None
    Notebook への表示（IPython.display.display）:
        1) Raw JSON（整形表示）
        2) 出力補足テーブル（フィールド / 値 / 説明）
           - JSONの出現順（dictの順序）を最優先して表示
           - list は「配列そのもの（prefix）」→「先頭要素の中身（prefix[]...）」の順で表示
           - choices[].message.content は表ではプレースホルダ表示し、本文は別枠で表示
           - 説明は FIELD_DESCRIPTIONS を優先し、未登録は auto_description() で自動生成
        3) choices[].message.content（回答本文）を Markdown 表示（箇条書き等を維持）

仕様メモ:
    - 「JSON順」を崩さないため、並べ替えは行いません（discover_paths_in_order の順を採用）。
    - もし JSON に無いが強制的に表示したいパスがある場合のみ FIELD_ORDER_FALLBACK に追加できます。
"""

import json
from typing import Any, Dict, List, Optional, Tuple, Set, Union
from IPython.display import Markdown, HTML, display


# 「もしJSONに無いが、表示したい」ものがあればここに入れる（基本は少なめ推奨）
# ※ JSON順を崩さないため、このリストは “不足分だけ末尾に補う” ために使う
FIELD_ORDER_FALLBACK: List[str] = [
    # 例: "usage.total_tokens",
]

# 既知フィールドの説明（無ければ自動生成）
FIELD_DESCRIPTIONS: Dict[str, str] = {
    # --- Top-level ---
    "id": "このレスポンスの一意な識別子（追跡・問い合わせ用）",
    "created": "レスポンス生成時刻（UnixTime: 秒）",
    "model": "実際に応答生成に使われたモデル名（バージョン含むことあり）",
    "object": "レスポンス種別（このAPIが返すオブジェクトタイプ）",
    "service_tier": "処理レーン/サービス階層（default 等）",
    "system_fingerprint": "モデル実装の指紋（同一モデルでも内部更新を識別する用途）",

    # --- choices ---
    "choices": "生成結果の候補配列（通常 choices[0] を採用）",
    "choices[].finish_reason": "生成が止まった理由（stop/length/tool_calls 等）",
    "choices[].index": "choices 配列内の候補番号（0始まり）",
    "choices[].logprobs": "トークン確率情報（有効化したときだけ付く）",

    # --- message ---
    "choices[].message": "生成メッセージ本体（role/content 等）",
    "choices[].message.role": "発話者ロール（assistant 等）",
    "choices[].message.content": "生成テキスト本文（表では別枠表示）",
    "choices[].message.refusal": "拒否の理由/種別（拒否時のみ。通常 null）",
    "choices[].message.annotations": "注釈情報（機能/モデルにより付与）",
    "choices[].message.audio": "音声出力メタ情報（音声出力時のみ）",
    "choices[].message.function_call": "旧式の関数呼び出し情報（互換用。通常 null）",
    "choices[].message.tool_calls": "ツール呼び出し情報（ツール利用時のみ）",

    # --- usage ---
    "usage": "トークン使用量の集計（課金/最適化の基本）",
    "usage.prompt_tokens": "入力側トークン数（プロンプトの長さ）",
    "usage.completion_tokens": "出力側トークン数（回答の長さ）",
    "usage.total_tokens": "合計トークン数（prompt + completion）",

    # --- usage details (prompt) ---
    "usage.prompt_tokens_details": "入力トークンの内訳（対応時のみ）",
    "usage.prompt_tokens_details.cached_tokens": "キャッシュヒットした入力トークン数（該当時）",
    "usage.prompt_tokens_details.audio_tokens": "入力の音声トークン数（音声入力時）",

    # --- usage details (completion) ---
    "usage.completion_tokens_details": "出力トークンの内訳（対応時のみ）",
    "usage.completion_tokens_details.reasoning_tokens": "推論に割り当てられたトークン数（対応時）",
    "usage.completion_tokens_details.accepted_prediction_tokens": "採用された予測トークン数（対応時）",
    "usage.completion_tokens_details.rejected_prediction_tokens": "棄却された予測トークン数（対応時）",
    "usage.completion_tokens_details.audio_tokens": "出力の音声トークン数（音声出力時）",
}



def _to_dict(response_obj: Union[Any, Dict[str, Any]]) -> Dict[str, Any]:
    """
    response_obj を dict に正規化する。

    Args:
        response_obj:
            - dict または model_dump() を持つオブジェクト

    Returns:
        dict 化したレスポンス

    Raises:
        TypeError: model_dump() を持たず dict でもない場合
    """
    if isinstance(response_obj, dict):
        return response_obj
    model_dump = getattr(response_obj, "model_dump", None)
    if callable(model_dump):
        return model_dump()
    raise TypeError("response_obj は model_dump() を持つオブジェクトか dict である必要があります。")


def display_raw_json(data: Dict[str, Any]) -> None:
    """
    レスポンス全体を整形JSONとして表示する（Markdownコードブロック）。

    Args:
        data: レスポンスdict
    """
    pretty = json.dumps(data, indent=2, ensure_ascii=False)
    display(Markdown("## 🧾 Raw JSON"))
    display(Markdown(f"```json\n{pretty}\n```"))


def escape_html(s: Any) -> str:
    """
    HTML表示用の最低限のエスケープ。

    Args:
        s: 任意の値

    Returns:
        HTMLエスケープ済み文字列
    """
    if s is None:
        return ""
    s = str(s)
    return s.replace("&", "&amp;").replace("<", "&lt;").replace(">", "&gt;")


def format_value_for_table(v: Any, max_len: Optional[int] = 180) -> str:
    """
    テーブルの「値」表示用に整形する（HTML文字列）。

    仕様:
        - None は "null" と表示（JSONの見た目に合わせる）
        - dict/list は JSON 文字列化（ensure_ascii=False）
        - 改行 → <br>
        - HTMLエスケープ（ただし <br> は復活）
        - 長文は max_len で省略（None なら省略しない）

    Args:
        v: 表示する値
        max_len: 最大表示文字数（Noneなら省略しない）

    Returns:
        HTMLとして安全な表示用文字列
    """
    if v is None:
        s = "null"  # ★ここが修正点（空にしない）
    elif isinstance(v, (dict, list)):
        # JSONっぽく見せる（Pythonの None/True/False を出さない）
        s = json.dumps(v, ensure_ascii=False)
    elif isinstance(v, bool):
        # JSON表記に寄せる（true/false）
        s = "true" if v else "false"
    else:
        s = str(v)

    s = s.replace("\n", "<br>")
    s = escape_html(s).replace("&lt;br&gt;", "<br>")  # <br>だけ復活

    if max_len is not None and len(s) > max_len:
        s = s[:max_len] + "…"
    return s


def get_value_by_path(data: Any, path: str) -> Any:
    """
    path で指定された値を取得する。

    ルール:
        - "choices[]" のような [] は配列の先頭要素を代表として扱う
          例: choices[].message.role は choices[0].message.role を参照する

    Args:
        data: ルートオブジェクト（dict想定だが、途中でlist/scalarもありうる）
        path: ドット区切りパス（例: "usage.total_tokens", "choices[].message.content"）

    Returns:
        該当値（存在しない/辿れない場合は None）
    """
    cur = data
    for part in path.split("."):
        is_list = part.endswith("[]")
        key = part[:-2] if is_list else part

        if isinstance(cur, list):
            cur = cur[0] if cur else None

        if cur is None:
            return None
        if not isinstance(cur, dict):
            return None

        cur = cur.get(key)

        if is_list and isinstance(cur, list):
            cur = cur[0] if cur else None
    return cur


def discover_paths_in_order(data: Any, prefix: str = "") -> List[str]:
    """
    JSONの出現順（dictの順序）をできるだけ維持してフィールドパスを収集する。

    ルール:
        - dict: キー順に再帰（prefix.key）
        - list:
            1) prefix（配列そのもの）を先に paths に追加
            2) 先頭要素があれば prefix[] を付けて先頭要素を再帰（中身のフィールドを拾う）
        - scalar: prefix を追加

    Args:
        data: 任意のJSONライクなデータ（dict/list/scalar）
        prefix: 現在のパス

    Returns:
        フィールドパスのリスト（JSON順に近い）
    """
    paths: List[str] = []

    if isinstance(data, dict):
        for k, v in data.items():
            p = f"{prefix}.{k}" if prefix else k
            paths.extend(discover_paths_in_order(v, p))
        return paths

    if isinstance(data, list):
        if prefix:
            paths.append(prefix)  # 配列そのもの
            if data:
                paths.extend(discover_paths_in_order(data[0], prefix + "[]"))
        return paths

    if prefix:
        paths.append(prefix)
    return paths


def auto_description(path: str, value: Any) -> str:
    """
    FIELD_DESCRIPTIONS に無いフィールドの説明を自動生成する。

    方針:
        - 末尾キー名（role/content/tokens 等）でなるべく意味のある説明を付ける
        - それでも難しい場合は型（None/int/str/list/dict）から推測

    Args:
        path: フィールドパス
        value: 実値（Noneの可能性あり）

    Returns:
        自動生成した説明文
    """
    last = path.split(".")[-1]
    if last.endswith("[]"):
        last = last[:-2]

    # キー名ベース（よくあるものを優先）
    if last == "content":
        return "本文テキスト"
    if last == "role":
        return "発話者ロール"
    if last == "index":
        return "配列のインデックス"
    if last == "refusal":
        return "拒否に関する情報（通常は null）"
    if last == "annotations":
        return "注釈情報（通常は空/なし）"
    if last == "audio":
        return "音声関連情報（音声入出力時のみ）"
    if last in ("function_call", "tool_calls"):
        return "関数/ツール呼び出し情報（該当時のみ）"
    if last.endswith("_tokens"):
        return "トークン数"
    if last.endswith("_token"):
        return "トークン"

    # 型ベース
    if value is None:
        return "（値なし / null）"
    if isinstance(value, bool):
        return "真偽値"
    if isinstance(value, int):
        return "数値"
    if isinstance(value, float):
        return "数値（小数）"
    if isinstance(value, str):
        return "文字列"
    if isinstance(value, list):
        return "配列"
    if isinstance(value, dict):
        return "オブジェクト"

    return "（自動説明）"


def build_rows_json_order(data: Dict[str, Any]) -> Tuple[List[Tuple[str, str, str]], str]:
    """
    JSON順を最優先にテーブル行を作る。

    仕様:
        - discover_paths_in_order の順番をそのまま採用（並べ替えなし）
        - FIELD_ORDER_FALLBACK は「JSONに無いが表示したい」場合のみ末尾に補う
        - choices[].message.content は表ではプレースホルダ、本文として別返却

    Args:
        data: レスポンスdict

    Returns:
        rows: (field_path, value_html, description) のリスト
        content_text: choices[].message.content の生本文（無ければ ""）
    """
    discovered = discover_paths_in_order(data)
    discovered_set: Set[str] = set(discovered)

    ordered = list(discovered)
    for p in FIELD_ORDER_FALLBACK:
        if p not in discovered_set:
            ordered.append(p)

    rows: List[Tuple[str, str, str]] = []
    content_text = ""

    for p in ordered:
        val = get_value_by_path(data, p)

        desc = FIELD_DESCRIPTIONS.get(p)
        if desc is None:
            desc = auto_description(p, val)

        if p == "choices[].message.content":
            if isinstance(val, str):
                content_text = val
            value_html = "（下に本文を表示）"
        else:
            value_html = format_value_for_table(val)

        rows.append((p, value_html, desc))

    return rows, content_text


def render_html_table(rows: List[Tuple[str, str, str]]) -> None:
    """
    補足テーブルを HTML で表示する。

    表示仕様:
        - フィールド列は左寄せ + monospace
        - ゼブラ表示
        - 横スクロール対応

    Args:
        rows: (field, value_html, desc) の行リスト
    """
    html: List[str] = []
    html.append('<h2>📘 出力補足（JSON順・動的）</h2>')
    html.append('<div style="overflow-x:auto;">')
    html.append('<table style="border-collapse:collapse; width:100%;">')
    html.append("<thead><tr>")
    html.append('<th style="text-align:left; padding:8px 10px; border-bottom:1px solid #ddd;">フィールド</th>')
    html.append('<th style="text-align:left; padding:8px 10px; border-bottom:1px solid #ddd;">値</th>')
    html.append('<th style="text-align:left; padding:8px 10px; border-bottom:1px solid #ddd;">説明</th>')
    html.append("</tr></thead><tbody>")

    zebra = False
    for field, value_html, desc in rows:
        bg = "#fafafa" if zebra else "white"
        zebra = not zebra
        html.append(f'<tr style="background:{bg};">')
        html.append(
            f'<td style="text-align:left; vertical-align:top; padding:8px 10px; font-family:monospace;">'
            f'{escape_html(field)}</td>'
        )
        html.append(f'<td style="vertical-align:top; padding:8px 10px;">{value_html}</td>')
        html.append(f'<td style="vertical-align:top; padding:8px 10px;">{escape_html(desc)}</td>')
        html.append("</tr>")

    html.append("</tbody></table></div>")
    display(HTML("\n".join(html)))


def display_content_block(content: str) -> None:
    """
    choices[].message.content（回答本文）を Markdown 表示する。

    Args:
        content: 本文文字列
    """
    display(Markdown("## ✨ choices[].message.content（回答本文）"))
    display(Markdown(content or ""))


def display_response_analysis_json_order(response_obj: Union[Any, Dict[str, Any]]) -> None:
    """
    エントリーポイント。

    入力:
        response_obj: Any | dict
            - model_dump() を持つオブジェクト、または dict

    出力:
        戻り値 None
        Notebook 表示:
            1) Raw JSON
            2) JSON順の動的補足テーブル
            3) 本文（content）

    Args:
        response_obj: レスポンスオブジェクト
    """
    data = _to_dict(response_obj)

    display_raw_json(data)

    rows, content = build_rows_json_order(data)
    render_html_table(rows)

    display_content_block(content)


# ================================
# 実行
# ================================
display_response_analysis_json_order(response)


In [ ]:
# 言語モデルからの回答を表示（Markdown形式）
from IPython.display import Markdown, display

# 役割や前提の設定
role = "あなたはマーケティング分野に精通したビジネスコンサルタントです。企業の成長をサポートするために、効果的なマーケティング戦略を提供します。"

# メッセージの設定
message = "新製品の発売に向けた効果的なマーケティング戦略を教えてください。"

# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": role},
        {"role": "user", "content": message},
    ],
)

# 言語モデルからの回答を表示
display(Markdown(response.choices[0].message.content.strip()))

In [ ]:
from IPython.display import Markdown, display

# 出力の多様性
message = "日本の魅力を紹介して！"

response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[{"role": "user", "content": message}],
    temperature=1.4,
    # temperature:
    #   「生成のランダムさ（どれくらい冒険するか）」を調整するパラメータ。
    #
    # --------------------------------------------------
    # ■ まず前提：LLMは“確率分布”から次トークンを選んでいる
    # --------------------------------------------------
    # モデルは次の単語（正確にはトークン）を出すとき、
    #
    #   1) すべての候補トークンにスコア（logit）を付ける
    #   2) softmax関数で確率分布に変換する
    #   3) その確率に従って1つ選ぶ
    #
    # という流れで動いている。
    #
    # temperature は、この「確率分布の尖り具合」を調整する。
    #
    #
    # --------------------------------------------------
    # ■ 数学的には何をしているの？
    # --------------------------------------------------
    # softmaxに入れる前のlogitを
    #
    #     logit / temperature
    #
    # のように割る処理が入る。
    #
    # これが何を意味するかというと：
    #
    #   temperature が小さい → 分布が“尖る”
    #   temperature が大きい → 分布が“平らになる”
    #
    #
    # --------------------------------------------------
    # ■ 直感的なイメージ
    # --------------------------------------------------
    # temperature = 0 に近い
    #   → ほぼ一番確率の高い単語しか選ばない
    #   → 毎回ほぼ同じ出力になる
    #   → 安定・保守的・創造性低い
    #
    # temperature = 1.0
    #   → モデル本来の確率分布そのまま
    #   → バランス型（デフォルト相当）
    #
    # temperature > 1.0
    #   → 低確率の単語も選ばれやすくなる
    #   → 予想外の表現が増える
    #   → 創造的だが、不安定にもなる
    #
    #
    # --------------------------------------------------
    # ■ 今回の temperature = 1.4 はどういう状態？
    # --------------------------------------------------
    # 1.4 は「やや高め」。
    #
    # 意味としては：
    #   - 普通よりもランダム性を強める
    #   - 意外性のある単語が出やすくなる
    #   - 会話が少しカオスになる可能性がある
    #
    # 例：
    #   temperature=0.2 → 無難な会話
    #   temperature=1.0 → 自然な会話
    #   temperature=1.4 → 少し飛んだ発言が出やすい
    #   temperature=2.0 → かなり予測不能
    #
    #
    # --------------------------------------------------
    # ■ なぜ高くすると“創造的”になるの？
    # --------------------------------------------------
    # 例えば次の候補確率があったとする：
    #
    #   こんにちは  70%
    #   やあ         20%
    #   ういっす       5%
    #   伝説的挨拶      0.1%
    #
    # temperature を上げると、
    #   70% が少し下がり、
    #   5% や 0.1% が相対的に上がる。
    #
    # その結果、
    #   普段は出ない語が出る可能性が増える。
    #
    #
    # --------------------------------------------------
    # ■ 注意点（重要）
    # --------------------------------------------------
    # temperature を上げすぎると：
    #
    #   - 文脈と合わない単語が出る
    #   - 話が急に飛ぶ
    #   - 一貫性が崩れる
    #   - 意味不明な文章になることもある
    #
    # つまり「創造的」と「不安定」は紙一重。
    #
    #
    # --------------------------------------------------
    # ■ よくある誤解
    # --------------------------------------------------
    # × temperature は“文章の長さ”を変える
    #   → 直接は関係ない（長さは max_tokens）
    #
    # × temperature は“知識の量”を変える
    #   → 知識量はモデル依存
    #
    # ○ temperature は“選択のランダムさ”を変える
    #
    #
    # --------------------------------------------------
    # ■ presence_penalty や logit_bias との関係
    # --------------------------------------------------
    # temperature は「確率分布全体の形」を変える。
    # logit_bias や penalty は「特定トークンのスコア」を変える。
    #
    # つまり：
    #   temperature = 全体のランダムさ
    #   logit_bias = 個別トークンの重み操作
    #   presence_penalty = 既出トークンへの補正
    #
    #
    # --------------------------------------------------
    # ■ まとめ
    # --------------------------------------------------
    # temperature = 1.4 は：
    #
    #   「やや攻めた生成設定」
    #
    #   - 創造性は上がる
    #   - 予測不能さも上がる
    #   - 会話が面白くなることもあるが、
    #     安定性は少し下がる
    #
    # 会話生成や物語では使えるが、
    # 正確性が重要なタスクでは控えめが無難。
    n=3, # 3回答
    max_completion_tokens=100
    # max_completion_tokens:
    #   モデルが「生成してよい最大トークン数（上限）」を指定するパラメータ。
    #
    #   ここでの 100 は「100文字」ではなく「100トークン」。
    #   トークンは単語やその一部、記号などの単位で、
    #   日本語では必ずしも「1文字＝1トークン」にはならない点に注意。
    #
    #   この値は“上限”なので、
    #   - 文章が自然に終われば100未満で止まる
    #   - まだ続きそうでも100に達したらそこで打ち切られる
    #
    #   長めの文章を書かせたい場合は増やし、
    #   短く要約させたい場合は小さくするなど、
    #   出力のボリュームをコントロールするために使う。
    #
    #   なお、入力トークン数とは別管理で、
    #   これは「出力側（completion）」だけの制限。
)

# すべてMarkdown表示
for i, choice in enumerate(response.choices, start=1):
    display(Markdown(f"## ✨ 回答 {i}"))
    display(Markdown(choice.message.content.strip()))


In [ ]:
from IPython.display import Markdown, display

# 出力の多様性
message = "日本の魅力を紹介して！"

response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[{"role": "user", "content": message}],
    presence_penalty=-2.0,
    # presence_penalty:
    #   「すでに一度でも出現したトークンを、その後どれくらい出にくく / 出やすくするか」
    #   を調整するパラメータ。
    #
    # --------------------------------------------
    # ■ まず前提：LLMは“次のトークン”を確率で選んでいる
    # --------------------------------------------
    # モデルは毎ステップ、
    #   1) 全トークン候補にスコア（logit）をつける
    #   2) softmaxで確率に変換する
    #   3) その確率に従って1つ選ぶ
    #
    # という処理をしている。
    #
    # presence_penalty は、
    #   「すでに出現したことがあるトークン」に対して
    #   そのスコアを調整する仕組み。
    #
    # --------------------------------------------
    # ■ presence_penalty の意味
    # --------------------------------------------
    #   正の値（例: 0.5, 1.0, 2.0）
    #       → 既出トークンを“出にくくする”
    #       → 同じ単語の繰り返しを抑える
    #       → 話題が広がりやすくなる
    #
    #   負の値（例: -0.5, -1.0, -2.0）
    #       → 既出トークンを“出やすくする”
    #       → 同じ語や話題を繰り返しやすくなる
    #       → 会話がループ気味になることもある
    #
    # --------------------------------------------
    # ■ 今回の -2.0 はどういう意味？
    # --------------------------------------------
    # presence_penalty = -2.0
    #
    # これはかなり強い「繰り返し促進」設定。
    #
    # つまり：
    #   一度でも出たトークンを、
    #   その後さらに“出やすく”する。
    #
    # 結果として：
    #   - 同じ名前を何度も出す
    #   - 同じ言い回しを繰り返す
    #   - 会話がやや単調になる
    #   - ループ的な文章になる可能性がある
    #
    # 通常は「繰り返しを減らしたい」ために
    # presence_penalty は正にすることが多い。
    #
    # --------------------------------------------
    # ■ frequency_penalty との違い（超重要）
    # --------------------------------------------
    # presence_penalty:
    #   → 「出現したかどうか」だけを見る（0回か1回以上か）
    #
    # frequency_penalty:
    #   → 「何回出たか」を見る（回数に応じて抑制）
    #
    # 例:
    #   Aという単語が5回出た場合
    #
    #   presence_penalty:
    #       → 1回でも出たら同じペナルティ
    #
    #   frequency_penalty:
    #       → 5回分のペナルティがかかる
    #
    # --------------------------------------------
    # ■ なぜ負にするの？
    # --------------------------------------------
    # 特殊用途として：
    #   - 同じキーワードを強く出し続けたい
    #   - 韻を踏ませたい
    #   - 強い反復表現を作りたい
    #   - 実験的に生成傾向を観察したい
    #
    # ただし実務では、
    #   -2.0 はかなり極端な値。
    #
    # 通常の自然な文章生成では
    #   0.0（デフォルト）〜 0.5 くらいが多い。
    #
    # --------------------------------------------
    # ■ まとめ
    # --------------------------------------------
    # presence_penalty = -2.0 は、
    #   「一度出た単語を、さらに出やすくする強めの設定」
    #
    # 文章は繰り返しが増えやすくなる。
    n=3, # 3回答
    max_completion_tokens=300  # 100だと途中で切れたので300に増やしました
)

# 結果をMarkdown形式で表示
for i, choice in enumerate(response.choices, start=1):
    display(Markdown(f"## ✨ 回答 {i}"))
    display(Markdown(choice.message.content.strip()))
    display(Markdown(f"**finish_reason:** `{choice.finish_reason}`"))
    display(Markdown("---"))


In [ ]:
import tiktoken
from tiktoken.core import Encoding

# OpenAI APIの特定のモデルに対応するトークナイザーを取得
encoding: Encoding = tiktoken.encoding_for_model(MODEL_NAME) # o200k_base

# テキストをトークンIDのリストに変換
tokens = encoding.encode("こんにちは！")
tokens_count = len(tokens)

# トークンの長さとトークンIDを表示
print(f"{tokens_count=}")
print(f"{tokens=}")

In [ ]:
# =========================
# トークンの選択度合い（プロンプト）
# =========================
message = """
AさんとBさんで会話してください。
A:あ、Bさんだ。
B:
"""
# ↑ この文字列が「ユーザーからモデルへの指示」になる。
#
# LLM(大規模言語モデル)は、文章を “文字” ではなく “トークン” という単位に分解して扱う。
# トークンはだいたい「単語」「単語の一部」「記号」「改行」などが混ざった粒度。
#
# このプロンプトは、
#   - 「AさんとBさんで会話して」と命令し
#   - すでに A の発話が1つ書いてあり
#   - B: の後をモデルに続けさせる
# という形。
#
# つまり「B:」の続きをモデルが生成し、その後も会話っぽく続けることが期待される。

response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": message},
    ],
    # messages:
    #   Chat Completions は「会話履歴」を配列で渡す形式。
    #   各要素は {role, content} を持つ。
    #
    #   role には主に:
    #     - "system": モデル全体の振る舞い方針（最優先ルール）
    #     - "user": ユーザー発言
    #     - "assistant": これまでのモデル出力（会話の続きを作るために渡すことがある）
    #
    #   今回は "user" のみで、会話履歴は1発目の指示だけ。
    max_tokens=100,
    # max_tokens:
    #   「モデルが生成してよいトークン数の上限」。
    #
    #   重要: 100 “文字” ではなく 100 “トークン”。
    #   日本語はトークン化のされ方が英語と違うので、
    #   体感として「100トークン = 100文字」にはならない。
    #
    #   上限なので、途中で会話が自然に終わったら100未満で止まることもある。
    #   逆に会話が続きそうでも 100 に達したらそこで強制終了する。
    n=3,
    # n:
    #   「同じプロンプトに対して、生成結果を何個返すか」。
    #
    #   n=3 にすると、モデルは “別々の候補” を3つ返す。
    #   （同じ指示でも文章生成は確率的なので、毎回少し違う結果になる）
    #
    #   使いどころ:
    #     - 文章のバリエーションが欲しい
    #     - どれが良いか選びたい
    #     - 生成がブレやすいタスクで保険をかけたい
    #
    #   注意:
    #     - n を増やすほど生成コストも増えやすい（生成が複数回走るため）
    #     - 返ってくる response.choices の数が増える
    logit_bias={95839: 8, 3393: 8}
    # logit_bias:
    #   ここが一番 “LLM初心者が意味不明になりがち” なところなので超詳しく書く。
    #
    # ■ そもそも「logit」って何？
    #   モデルは次のトークンを出すとき、内部で
    #     「候補トークンごとの点数（スコア）」
    #   を計算する。この “生のスコア” が logit（ロジット）。
    #
    #   その後、logit を softmax という関数に通すことで
    #     「確率（0〜1の分布）」
    #   に変換し、その確率に従って次トークンを選ぶ。
    #
    #
    # ■ logit_bias は何をする？
    #   「特定のトークンの logit に加算（または減算）して、
    #    そのトークンが選ばれる確率を上げたり下げたりする」
    #   ための仕組み。
    #
    #   例:
    #     logit_bias = { token_id: bias_value }
    #
    #   bias_value が正なら → そのトークンが “出やすくなる”
    #   bias_value が負なら → そのトークンが “出にくくなる”
    #
    #   ここでは 8 を指定しているので、
    #   token_id=95839 と token_id=3393 の2つを「かなり出やすくする」設定。
    #
    #
    # ■ 「token_id」って何？ 95839 とか 3393 は何者？
    #   これは “トークナイザが割り当てたID”。
    #   モデルは文字列をトークン列に変換して処理するが、
    #   実際には各トークンは整数ID（token_id）として扱われる。
    #
    #   つまり {95839: 8} は
    #     「ID=95839 のトークンを強く出したい」
    #   という意味。
    #
    #   ただし注意点があって、
    #   token_id は “モデルやトークナイザ” で変わることがある。
    #   別モデルに変えたら同じIDが別文字列だったり、存在しない場合もある。
    #
    #
    # ■ bias=8 ってどれくらい強い？
    #   logit は確率に変換される前のスコアなので、直感が難しい。
    #   でも一般的に、
    #     +1〜+2: 少し出やすい
    #     +5: かなり効く
    #     +8: 強め（文脈によってはほぼ強制に近いことも）
    #   という体感になりやすい。
    #
    #   ただし、絶対にそのトークンが出るわけではない。
    #   “文脈に合わない” トークンは、+8しても出ない/出づらいこともある。
    #
    #
    # ■ どういうときに使う？
    #   - 禁止したい語を出にくくする（負のバイアス）
    #   - ある表記（例: 「株式会社」表記）を優先させたい
    #   - 特定の区切り記号を出しやすくしてフォーマットを安定させたい
    #
    #   ただし「文章の品質が上がる魔法」ではなく、
    #   下手に強くかけると不自然な文章になったり、同じ語が連発されたりする。
    #
    #
    # ■ このコードの意図（推測）
    #   {95839: 8, 3393: 8} の2つが何のトークンか次第だが、
    #   「会話の体裁（A: / B:）を安定させたい」とか
    #   「特定の語を出したい」目的で入れている可能性がある。
    #
    #   ※ “どの文字列に対応する token_id か” は、実際には
    #   使用モデルのトークナイザで確認しないと断定できない。
)

for choice in response.choices:
    # response.choices:
    #   n=3 を指定しているので choices が3件入ってくる想定。
    #
    #   それぞれは「別候補の生成結果」。
    #   choice.message.content に生成テキストが入る。

    print("-" * 20)
    # 区切り線を表示して、複数候補が見やすいようにする。

    print(choice.message.content.strip())
    # strip():
    #   生成テキストの前後に入りがちな改行や空白を取り除いて表示する。
    #   LLMは出力冒頭に改行を挟むことがよくあるので、見た目を整える定番テク。


In [ ]:
import time
from IPython.display import Markdown, display

message = "言語モデルを使う上でのポイントは"

stream = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[{"role": "user", "content": message}],
    stream=True, # ここで回答をストリーミングにできるらしい
)

text = ""
handle = display(Markdown(text), display_id=True)

last_update = 0.0
UPDATE_INTERVAL_SEC = 0.08  # 0.05〜0.2くらいで好み調整

for chunk in stream:
    if chunk.choices:
        delta = chunk.choices[0].delta.content
        if delta is not None:
            text += delta

            now = time.time()
            if now - last_update >= UPDATE_INTERVAL_SEC:
                handle.update(Markdown(text))
                last_update = now

# 最後に確実に最新状態を反映
handle.update(Markdown(text))
